In [1]:
import itertools

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy import stats, special
from statsmodels.sandbox.distributions.transformed import Transf_gen


 - https://en.wikipedia.org/wiki/Logit-normal_distribution
 - https://en.wikipedia.org/wiki/Logistic_function
 - https://en.wikipedia.org/wiki/Logit
 
 - https://github.com/scipy/scipy/blob/v1.8.1/scipy/stats/_continuous_distns.py#L5365-L5499
 
 - https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.rv_continuous.html?highlight=rv_continous
 - https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.expit.html
 - https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.logit.html
 
 - https://github.com/statsmodels/statsmodels/blob/main/statsmodels/sandbox/distributions/transformed.py#L55
 - https://github.com/scipy/scipy/issues/12133
 
![Example](https://upload.wikimedia.org/wikipedia/commons/a/ae/LogitnormDensityGrid.svg)
 

In [10]:
from scipy import special


class logitnorm_gen(stats.rv_continuous):
    
    def _pdf(self, x):
        return stats.norm.pdf(special.logit(x))/(x*(1-x))
    
    def _cdf(self, x):
        return stats.norm.cdf(special.logit(x))
    
    def _rvs(self, size=None, random_state=None):
        print(self.__dict__)
        return special.expit(random_state.standard_normal(size))
    
    def fit(self, data, **kwargs):
        data = special.logit(data)
        return stats.norm.fit(data, **kwargs)

    
logitnorm = logitnorm_gen(name="logitnorm", longname="Logit Normal", a=0.0, b=1.0)
#logitnorm = Transf_gen(
#    stats.norm, special.expit, special.expit, decr=False, a=0.0, b=1.0,
#    numargs = 0, name = 'discf', longname = 'normal-based discount factor',
#    extradoc = '\ndistribution of discount factor y=1/(1+x)) with x N(0.05,0.1**2)'
#)

In [11]:
logitnorm.support()

(0.0, 1.0)

In [12]:
logitnorm()

In [13]:
fixtures = []
for (mu, sigma) in itertools.product([0.0, 0.5, 1.0, 1.5, 2.0], [0.32, 0.56, 1.00, 1.78, 3.16]):
    fixtures.append({"parameters": {"loc": mu, "scale": sigma}, "size": 1000, "seed": 486})

In [14]:
for fixture in fixtures:
    
    # Create samples:
    normal = stats.norm(**fixture["parameters"])
    sample = normal.rvs(size=fixture["size"], random_state=fixture["seed"])
    reference = special.expit(sample)
    law = logitnorm(**fixture["parameters"])
    check = law.rvs(size=fixture["size"], random_state=fixture["seed"])
    
    print(fixture, np.allclose(reference - check, 0.0))
    
    # Show:
    if False:
        fig, axe = plt.subplots(1, 3, sharey=True)
        axe[0].hist(sample)
        axe[1].hist(reference)
        axe[2].hist(check, alpha=0.3)
        fig.suptitle("Logit Normal $\mu={loc:.2f}, \sigma={scale:.2f}$".format(**fixture["parameters"]))
        for i in range(3):
            axe[i].grid()
            if i > 0:
                axe[i].set_xlim([0, 1])

{'_stats_has_moments': True, '_random_state': RandomState(MT19937) at 0x7FA324E93780, '_rvs_uses_size_attribute': False, '_rvs_size_warned': False, '_ctor_param': {'momtype': 1, 'a': 0.0, 'b': 1.0, 'xtol': 1e-14, 'badvalue': nan, 'name': 'logitnorm', 'longname': 'Logit Normal', 'shapes': None, 'extradoc': None, 'seed': None}, 'badvalue': nan, 'name': 'logitnorm', 'a': 0.0, 'b': 1.0, 'xtol': 1e-14, 'moment_type': 1, 'shapes': None, '_parse_args': <bound method _parse_args of <__main__.logitnorm_gen object at 0x7fa2ffcfe320>>, '_parse_args_stats': <bound method _parse_args_stats of <__main__.logitnorm_gen object at 0x7fa2ffcfe320>>, '_parse_args_rvs': <bound method _parse_args_rvs of <__main__.logitnorm_gen object at 0x7fa2ffcfe320>>, 'numargs': 0, '_ppfvec': <numpy.vectorize object at 0x7fa2ffcfee10>, 'vecentropy': <numpy.vectorize object at 0x7fa2ffcfeb38>, '_cdfvec': <numpy.vectorize object at 0x7fa2ffcfe978>, 'extradoc': None, 'generic_moment': <numpy.vectorize object at 0x7fa2ffcfe1

{'_stats_has_moments': True, '_random_state': RandomState(MT19937) at 0x7FA324E93780, '_rvs_uses_size_attribute': False, '_rvs_size_warned': False, '_ctor_param': {'momtype': 1, 'a': 0.0, 'b': 1.0, 'xtol': 1e-14, 'badvalue': nan, 'name': 'logitnorm', 'longname': 'Logit Normal', 'shapes': None, 'extradoc': None, 'seed': None}, 'badvalue': nan, 'name': 'logitnorm', 'a': 0.0, 'b': 1.0, 'xtol': 1e-14, 'moment_type': 1, 'shapes': None, '_parse_args': <bound method _parse_args of <__main__.logitnorm_gen object at 0x7fa2ffd32358>>, '_parse_args_stats': <bound method _parse_args_stats of <__main__.logitnorm_gen object at 0x7fa2ffd32358>>, '_parse_args_rvs': <bound method _parse_args_rvs of <__main__.logitnorm_gen object at 0x7fa2ffd32358>>, 'numargs': 0, '_ppfvec': <numpy.vectorize object at 0x7fa2ffd323c8>, 'vecentropy': <numpy.vectorize object at 0x7fa2ffd32748>, '_cdfvec': <numpy.vectorize object at 0x7fa2ffd32390>, 'extradoc': None, 'generic_moment': <numpy.vectorize object at 0x7fa2ffd324

{'parameters': {'loc': 1.5, 'scale': 0.32}, 'size': 1000, 'seed': 486} False
{'_stats_has_moments': True, '_random_state': RandomState(MT19937) at 0x7FA324E93780, '_rvs_uses_size_attribute': False, '_rvs_size_warned': False, '_ctor_param': {'momtype': 1, 'a': 0.0, 'b': 1.0, 'xtol': 1e-14, 'badvalue': nan, 'name': 'logitnorm', 'longname': 'Logit Normal', 'shapes': None, 'extradoc': None, 'seed': None}, 'badvalue': nan, 'name': 'logitnorm', 'a': 0.0, 'b': 1.0, 'xtol': 1e-14, 'moment_type': 1, 'shapes': None, '_parse_args': <bound method _parse_args of <__main__.logitnorm_gen object at 0x7fa2ffd32358>>, '_parse_args_stats': <bound method _parse_args_stats of <__main__.logitnorm_gen object at 0x7fa2ffd32358>>, '_parse_args_rvs': <bound method _parse_args_rvs of <__main__.logitnorm_gen object at 0x7fa2ffd32358>>, 'numargs': 0, '_ppfvec': <numpy.vectorize object at 0x7fa2ffd32390>, 'vecentropy': <numpy.vectorize object at 0x7fa2ffd320b8>, '_cdfvec': <numpy.vectorize object at 0x7fa2ffd324e0>